## BBC News Classification

Completed as part of CU Boulder's Unsupervised Algorithms in Machine Learning course.

In [2]:
# Project dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

In [8]:
# Data import
train = pd.read_csv("data/BBC News Train.csv")
test = pd.read_csv("data/BBC News Test.csv")

### Exploratory Data Analysis (EDA)

In [9]:

print(train.head(5))
print("\nTrain:", train.info())
print("\nTest:", test.info())

   ArticleId                                               Text  Category
0       1833  worldcom ex-boss launches defence lawyers defe...  business
1        154  german business confidence slides german busin...  business
2       1101  bbc poll indicates economic gloom citizens in ...  business
3       1976  lifestyle  governs mobile choice  faster  bett...      tech
4        917  enron bosses in $168m payout eighteen former e...  business
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ArticleId  1490 non-null   int64 
 1   Text       1490 non-null   object
 2   Category   1490 non-null   object
dtypes: int64(1), object(2)
memory usage: 35.0+ KB

Train: None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Arti

In [11]:
# Check for any null/null-like values
null_like = [np.nan, None, [], {}, 'NaN', 'Null','NULL','None','NA','?','-', '.', '', ' ', '   ']

for df in [train, test]:
    for c in df.columns:
        string_null = np.array([x in null_like for x in df[c]])
        print(c, df[c].isnull().sum(), string_null.sum())

ArticleId 0 0
Text 0 0
Category 0 0
ArticleId 0 0
Text 0 0


In [13]:
# Vectorizing word data
# importing all necessary modules
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
warnings.filterwarnings(action = 'ignore')

import gensim
from gensim.models import Word2Vec


In [19]:
import nltk
# for installing missing corpora if needed
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [29]:
# iterate through each sentence in the file
data = []
for idx, article in train.iterrows():
    text = article['Text']
    for i in sent_tokenize(text):
        temp = []
        # tokenize the sentence into words
        for j in word_tokenize(i):
            temp.append(j.lower())
        data.append(temp)
print(data[0],data[20])

['worldcom', 'ex-boss', 'launches', 'defence', 'lawyers', 'defending', 'former', 'worldcom', 'chief', 'bernie', 'ebbers', 'against', 'a', 'battery', 'of', 'fraud', 'charges', 'have', 'called', 'a', 'company', 'whistleblower', 'as', 'their', 'first', 'witness', '.'] ['germany', 's', 'economy', 'grew', '1.6', '%', 'last', 'year', 'after', 'shrinking', 'in', '2003.', 'however', 'the', 'economy', 'contracted', 'by', '0.2', '%', 'during', 'the', 'last', 'three', 'months', 'of', '2004', 'mainly', 'due', 'to', 'the', 'reluctance', 'of', 'consumers', 'to', 'spend', '.']


In [34]:

model1 = gensim.models.Word2Vec(data, min_count = 1,
                              vector_size = 200, window = 5)

# Print results
print("Score 'lawyers' <-> 'charges' - CBOW : ", model1.wv.similarity('lawyers', 'defence'))

print("Score 'lawyers' <-> 'germany':", model1.wv.similarity('lawyers', 'economy'))

Cosine similarity between 'lawyers' and 'charges' - CBOW :  0.9482143
Cosine similarity between 'lawyers' and 'germany' - CBOW :  0.8058109


In [12]:
fig, a = plt.subplots(1,3)
fig.set_figheight(12)
fig.set_figwidth(14)
fig.tight_layout()

features = train.drop(columns=["ArticleId"])

for i, c in enumerate(features.columns):
  data = train[c]
  ax = a.flat[i]
  # ax.hist(data, bins=50, log=True)
  ax.set_title(c)

# for i in range(8,9):
#   a.flat[i].set_visible(False)

IndexError: index 8 is out of bounds for axis 0 with size 3

ValueError: Image size of 92468x872 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1008x864 with 3 Axes>